In [11]:
import torch
from torch import nn
from torch.nn import init
net = nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))
print(net)
X = torch.rand(2,4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [12]:
#access of model parameters
#以迭代器形式返回

In [13]:
print(type(net.named_parameters()))
for name,param in net.named_parameters():
    print(name,param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [14]:
for name, param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [15]:
class MyModule(nn.Module):
    def __init__(self,**kwargs):
        super(MyModule,self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20,20))
        self.weight2 = torch.rand(20,20)
    def forward(self,x):
        pass
n = MyModule()
for name,param, in n.named_parameters():
    print(name)

weight1


In [16]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[ 0.2388,  0.3390, -0.4588,  0.2756],
        [ 0.2037,  0.4874,  0.3877,  0.1161],
        [ 0.2979, -0.0100, -0.3552,  0.1207]])
None
tensor([[ 0.4235,  0.3566,  0.1375,  0.0881],
        [-0.3276, -0.2758, -0.1063, -0.0682],
        [ 0.1382,  0.1163,  0.0449,  0.0288]])


In [17]:
#initialization of model parameters

for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param,mean=0,std=0.01)
        print(name,param.data)

0.weight tensor([[ 8.8725e-03, -1.9340e-05, -6.1294e-03, -1.1040e-02],
        [-2.2151e-03, -5.8154e-03,  2.0339e-02, -7.9555e-03],
        [-9.2940e-03, -6.8782e-03,  5.2921e-03,  8.1787e-03]])
2.weight tensor([[-0.0147, -0.0044, -0.0007]])


In [18]:
#下面使用常数来初始化权重参数
for name,param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val=0)
        print(name,param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


In [ ]:
#自定义初始化方法

In [19]:
def normal_(tensor,mean=0,std=1):
    with torch.no_grad():
        return tensor.normal_(mean,std)

In [21]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor *= (tensor.abs() >= 5).float()
for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name,param.data)

0.weight tensor([[ 5.5096, -7.4900, -0.0000, -0.0000],
        [-9.8682,  0.0000, -9.8760, -5.9813],
        [ 0.0000,  0.0000, -5.4910,  6.2634]])
2.weight tensor([[-0.0000, 5.8324, 0.0000]])


In [22]:
for name,param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name,param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


In [23]:
#share of model parameter
linear = nn.Linear(1,1,bias=False)
net = nn.Sequential(linear,linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [24]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [25]:
x = torch.ones(1,1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
